In [3]:
import rtdl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# Load the data
import pandas as pd

train_path = '../../zipped/train.csv'
dev_in_path = '../../zipped/dev_in.csv'

df_train = pd.read_csv(train_path)
df_train.head()


,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0


In [7]:
df_dev_in = pd.read_csv(dev_in_path)
df_dev_in.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.1653,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0


In [18]:
# Identify the categorical features
cat_features = []
for col in df_dev_in:
    values = df_dev_in[col].tolist()
    unique = list(dict.fromkeys(values))
    if len(unique) < 20:
        cat_features.append(col)
print(cat_features)

['fact_cwsm_class', 'climate', 'cmc_available', 'gfs_available', 'gfs_soil_temperature_available', 'gfs_timedelta_s', 'wrf_available']


In [33]:
# !pip install scipy
from scipy import stats
import numpy as np

nan_replacements = {}
for col in df_train:
    if col in cat_features:
        nan_replacements[col] = stats.mode(np.asarray(df_train[col].tolist()))[0][0]
        # print(nan_replacements[col])
    else:
        nan_replacements[col] = np.mean(np.asarray(df_train[col].dropna().tolist()))
print(nan_replacements.values())

dict_values([1545219635.2602534, 27.3491997995972, -19.454721235367906, 15.030736293991083, 0.0, 'mild temperate', 309.93658796685, -2.7249874731371273, 15.053651434869764, 731.2756530851661, 289.1659257771938, 0.0513848381221949, 287.04881832634976, 287.0853175734476, 287.0339327353252, 258.9922501121575, 274.97475431188, 282.77351197349384, 285.8909051108904, 281.3276019215079, 7.660611531165116, 5.534873822269684, 14.704511863943937, 12.971739936778564, 8.78866233485156, 7.2775056043593676, 0.009529923266778687, 0.00859610917297107, 6.413124162323838, 0.2673820249969054, 0.027512247376225554, 0.008222988603064724, 3.866966535018241e-05, -0.0020003342605871715, -0.05948973513452053, 10.42531664950694, 5.2929354486362215, 1.8376276146288066, 0.4768480677913728, -0.09701087752828826, -0.14489588205685547, 0.22814633531193007, 0.2219977889627698, 0.30237136633108197, 0.18710502254215602, 97345.58898593165, 97344.2322319218, 101661.80541725569, 137.95607755299883, 5728.521789532874, 3094

In [34]:
# Replace nans in train and dev
for col in df_train:
    df_train[col] = df_train[col].fillna(nan_replacements[col])
    df_dev_in[col] = df_dev_in[col].fillna(nan_replacements[col])

In [35]:
# !pip install scikit-learn

from typing import Dict

import numpy as np
import sklearn.preprocessing


def normalize(
    X: Dict[str, np.ndarray], normalization: str, seed: int, noise: float = 1e-3
) -> Dict[str, np.ndarray]:
    # X ~ {'train': <train_size x n_features>, 'val': <val_size x n_features>, 'test': <test_size x n_features>}
    X_train = X['train']
    if normalization == 'standard':
        normalizer = sklearn.preprocessing.StandardScaler()
    elif normalization == 'quantile':
        normalizer = sklearn.preprocessing.QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X['train'].shape[0] // 30, 1000), 10),
            subsample=1e9,
            random_state=seed,
        )
        if noise:
            X_train = X_train.copy()
            stds = np.std(X_train, axis=0, keepdims=True)
            noise_std = noise / np.maximum(stds, noise)
            X_train += noise_std * np.random.default_rng(seed).standard_normal(
                X_train.shape
            )
    else:
        raise ValueError(f'unknown normalization: {normalization}')
    normalizer.fit(X_train)
    return {k: normalizer.transform(v) for k, v in X.items()}

In [36]:
# Set Seed
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [37]:
# Normalise using train data stats
# Quantile normalisation is used (maps to a normal distribution)
X_train_np = np.asarray(df_train.iloc[:,6:])
X_dev_in_np = np.asarray(df_dev_in.iloc[:,6:])
X = {'train': X_train_np, 'dev_in': X_dev_in_np}
X = normalize(X, normalization='quantile', seed=seed)
X_train_np = X['train']
X_dev_in_np = X['dev_in']

X_train = torch.FloatTensor(X_train_np)
X_dev_in = torch.FloatTensor(X_dev_in_np)

In [38]:
# Preprocess into tensors

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def get_lab_to_ind(data_df):
    '''
    Prepare a label to index map
    '''
    y_fact = set(list(data_df['fact_cwsm_class']))
    lab_to_ind = {}
    for i, lab in enumerate(y_fact):
        lab_to_ind[lab] = i
    return lab_to_ind

lab_to_ind = get_lab_to_ind(df_train)
batch_size = 1024

# Train
y_train = np.asarray(df_train['fact_cwsm_class'])
y_train = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_train]))

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Dev in
y_dev_in = df_dev_in['fact_cwsm_class']
y_dev_in = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_dev_in]))

dev_in_ds = TensorDataset(X_dev_in, y_dev_in)
dev_in_dl = DataLoader(dev_in_ds, batch_size=batch_size, shuffle=True)

In [39]:
# Get the device

def get_default_device():
    if torch.cuda.is_available():
        print("Got CUDA!")
        return torch.device('cuda')
    else:
        print("No CUDA found")
        return torch.device('cpu')

device = get_default_device()

Got CUDA!


In [40]:
# Create the Feature Transformer Model

model = rtdl.FTTransformer.make_default(
    n_num_features=X_train.shape[1],
    cat_cardinalities=None,
    last_layer_query_idx=[-1],
    d_out=len(lab_to_ind)
)

model.to(device)

FTTransformer(
  (feature_tokenizer): FeatureTokenizer(
    (num_tokenizer): NumericalFeatureTokenizer()
  )
  (cls_token): CLSToken()
  (transformer): Transformer(
    (blocks): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=192, out_features=192, bias=True)
          (W_k): Linear(in_features=192, out_features=192, bias=True)
          (W_v): Linear(in_features=192, out_features=192, bias=True)
          (W_out): Linear(in_features=192, out_features=192, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (ffn): FFN(
          (linear_first): Linear(in_features=192, out_features=512, bias=True)
          (activation): ReGLU()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_second): Linear(in_features=256, out_features=192, bias=True)
        )
        (attention_residual_dropout): Dropout(p=0.0, inplace=False)
        (ffn_residual_dropout): Dropout(p=0.0, inplace=False)


In [41]:
# Create optimizer

# These params don't need to be set as we will use a default optimizer
# lr = 0.01
# weight_decay = 0.9

optimizer = (
    model.make_default_optimizer()
    if isinstance(model, rtdl.FTTransformer)
    else torch.optim.AdamW(model.parameters(), lr=lr,
                          weight_decay=weight_decay)
)

In [42]:
# Create loss function criterion
criterion = nn.CrossEntropyLoss().to(device)

In [43]:
# Create pipeline to apply model
def apply_model(model, x_num, x_cat=None):
    '''
    FTTransformer expects numerical and categorical inputs separately
    '''
    return model(x_num, x_cat) if isinstance(model, rtdl.FTTransformer) else model(x_num)

In [45]:
# Training and Evaluation Function

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy_topk(output, target, k=1):
    """Computes the topk accuracy"""
    batch_size = target.size(0)

    _, pred = torch.topk(output, k=k, dim=1, largest=True, sorted=True)

    res_total = 0
    for curr_k in range(k):
      curr_ind = pred[:,curr_k]
      num_eq = torch.eq(curr_ind, target).sum()
      acc = num_eq/len(output)
      res_total += acc
    return res_total*100

def train(train_loader, model, criterion, optimizer, epoch, device, print_freq=100):
    '''
    Run one train epoch
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to train mode
    model.train()

    for i, (x, target) in enumerate(train_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = apply_model(model, x)
        loss = criterion(logits, target)

        # Backward pass and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'Accuracy {prec.val:.3f} ({prec.avg:.3f})'.format(
                      epoch, i, len(train_loader),
                      loss=losses, prec=accs))

@torch.no_grad()
def eval(val_loader, model, criterion, device):
    '''
    Run evaluation
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to eval mode
    model.eval()


    for i, (x, target) in enumerate(val_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = apply_model(model, x)
        loss = criterion(logits, target)

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

    print('Dev in\t Loss ({loss.avg:.4f})\t'
            'Accuracy ({prec.avg:.3f})\n'.format(
              loss=losses, prec=accs))

In [46]:
# Train
epochs = 10
for epoch in range(epochs):

    # train for one epoch
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(train_dl, model, criterion, optimizer, epoch, device)

    # evaluate on validation set
    eval(dev_in_dl, model, criterion, device)

current lr 1.00000e-04
Epoch: [0][0/3057]	Loss 2.2171 (2.2171)	Accuracy 4.590 (4.590)
Epoch: [0][100/3057]	Loss 1.0726 (1.2146)	Accuracy 56.055 (50.868)
Epoch: [0][200/3057]	Loss 1.0516 (1.1320)	Accuracy 56.152 (54.065)
Epoch: [0][300/3057]	Loss 1.0374 (1.0955)	Accuracy 56.348 (55.329)
Epoch: [0][400/3057]	Loss 1.0069 (1.0721)	Accuracy 58.984 (56.128)
Epoch: [0][500/3057]	Loss 0.9193 (1.0581)	Accuracy 62.402 (56.545)
Epoch: [0][600/3057]	Loss 0.9602 (1.0468)	Accuracy 59.961 (56.899)
Epoch: [0][700/3057]	Loss 0.9750 (1.0379)	Accuracy 59.668 (57.212)
Epoch: [0][800/3057]	Loss 0.9383 (1.0318)	Accuracy 61.523 (57.415)
Epoch: [0][900/3057]	Loss 0.9990 (1.0262)	Accuracy 59.277 (57.579)
Epoch: [0][1000/3057]	Loss 0.9861 (1.0214)	Accuracy 57.812 (57.752)
Epoch: [0][1100/3057]	Loss 0.9398 (1.0172)	Accuracy 60.840 (57.905)
Epoch: [0][1200/3057]	Loss 0.9479 (1.0136)	Accuracy 60.742 (58.044)
Epoch: [0][1300/3057]	Loss 0.9662 (1.0104)	Accuracy 59.277 (58.166)
Epoch: [0][1400/3057]	Loss 0.9452 (1.00

Epoch: [3][2600/3057]	Loss 0.9067 (0.9154)	Accuracy 62.891 (62.023)
Epoch: [3][2700/3057]	Loss 0.9372 (0.9154)	Accuracy 61.914 (62.027)
Epoch: [3][2800/3057]	Loss 0.9391 (0.9153)	Accuracy 59.766 (62.036)
Epoch: [3][2900/3057]	Loss 0.9274 (0.9151)	Accuracy 63.281 (62.045)
Epoch: [3][3000/3057]	Loss 0.9087 (0.9151)	Accuracy 62.402 (62.051)
Dev in	 Loss (0.8960)	Accuracy (62.854)

current lr 1.00000e-04
Epoch: [4][0/3057]	Loss 0.9282 (0.9282)	Accuracy 62.305 (62.305)
Epoch: [4][100/3057]	Loss 0.8957 (0.9130)	Accuracy 63.672 (62.301)
Epoch: [4][200/3057]	Loss 0.8792 (0.9098)	Accuracy 63.770 (62.402)
Epoch: [4][300/3057]	Loss 0.9082 (0.9091)	Accuracy 61.523 (62.376)
Epoch: [4][400/3057]	Loss 0.9404 (0.9081)	Accuracy 62.012 (62.398)
Epoch: [4][500/3057]	Loss 0.9263 (0.9075)	Accuracy 61.426 (62.383)
Epoch: [4][600/3057]	Loss 0.8960 (0.9079)	Accuracy 63.574 (62.354)
Epoch: [4][700/3057]	Loss 0.8933 (0.9080)	Accuracy 64.453 (62.352)
Epoch: [4][800/3057]	Loss 0.8865 (0.9081)	Accuracy 63.477 (62.

Epoch: [7][2000/3057]	Loss 0.9028 (0.8869)	Accuracy 62.012 (63.353)
Epoch: [7][2100/3057]	Loss 0.8261 (0.8869)	Accuracy 66.211 (63.356)
Epoch: [7][2200/3057]	Loss 0.8977 (0.8869)	Accuracy 62.402 (63.353)
Epoch: [7][2300/3057]	Loss 0.8765 (0.8869)	Accuracy 63.477 (63.345)
Epoch: [7][2400/3057]	Loss 0.9095 (0.8869)	Accuracy 62.207 (63.348)
Epoch: [7][2500/3057]	Loss 0.8224 (0.8868)	Accuracy 63.965 (63.349)
Epoch: [7][2600/3057]	Loss 0.8899 (0.8867)	Accuracy 64.160 (63.353)
Epoch: [7][2700/3057]	Loss 0.8790 (0.8866)	Accuracy 64.551 (63.360)
Epoch: [7][2800/3057]	Loss 0.8412 (0.8864)	Accuracy 66.797 (63.364)
Epoch: [7][2900/3057]	Loss 0.9311 (0.8864)	Accuracy 60.352 (63.363)
Epoch: [7][3000/3057]	Loss 0.9009 (0.8864)	Accuracy 63.086 (63.364)
Dev in	 Loss (0.8752)	Accuracy (63.830)

current lr 1.00000e-04
Epoch: [8][0/3057]	Loss 0.9032 (0.9032)	Accuracy 62.500 (62.500)
Epoch: [8][100/3057]	Loss 0.8777 (0.8832)	Accuracy 63.184 (63.433)
Epoch: [8][200/3057]	Loss 0.8365 (0.8834)	Accuracy 66.40